# Part 1: Database and Jupyter Set Up

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
# The default port used by MongoDB is 27017
mongo = MongoClient(port=27017)

In [ ]:
# Make sure to import json file and create a database called uk_food and a collection called establsihments
# mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json

In [3]:
# Examine available databases
print(mongo.list_database_names())

['admin', 'classDB', 'config', 'fruits_db', 'local', 'met', 'petsitly_marketing', 'travel_db', 'uk_food']


In [4]:
# Assign the uk_food database to a variable name
uk_food = mongo['uk_food']

In [5]:
# Review the collections in the database
print(uk_food.list_collection_names())

['establishments']


In [6]:
# assign collection to a variable
establishments = uk_food['establishments']

In [ ]:
# Review a document in the collection 
pprint(establishments.find_one())

# Part 2: Update the Database

## (1) Insert the halal restaurant to the uk_food database

In [10]:
# Build a dictionary for the new restuarant to add to the database
halal_rest = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [11]:
# Insert the new restaurant to the collection
establishments.insert_one(halal_rest)

In [12]:
# Check that the new restaurant was added; print "True" if the query for the new restaurant turns up empty
print(establishments.find({"BusinessName":"Penang Flavours"}) == None)

False


## (2) Find the BusinessTypeID for restaurants

In [23]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {"BusinessType":"Restaurant/Cafe/Canteen"}
fields = {'BusinessTypeID':1, 'BusinessType':1}

results_business_type = list(establishments.find(query, fields))

In [26]:
# Print the results
pprint(results_business_type[0])

{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('63cdeb79a0392a1bef670ac6')}


## (3) Update the BusinessTypeID for the new halal restaurant

In [27]:
# Update Penang Flavors with the correct business type 
uk_food.establishments.update_one(
    {"BusinessName":"Penang Flavours"},
    {'$set': 
        {'BusinessTypeID': 1}}
)

In [29]:
# Confirm that the new restaurant was updated
pprint(establishments.find({"BusinessName":"Penang Flavours"})[0])

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('63cdf125cfceb4e3db666072'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


## (4) Remove any establishments with Dover Local Authority

### Find how many documents have LocalAuthorityName as "Dover"

In [30]:
query = {'LocalAuthorityName': 'Dover'}
len(list(establishments.find(query)))

994

### Delete all documents with the 'LocalAuthorityName' as 'Dover'

In [31]:
uk_food.establishments.delete_many(query)

### Check if any other 'Dover' documents still exists

In [32]:
# Check how many documents exists still with 'Dover' as its 'LocalAuthorityName
len(list(establishments.find(query)))

0

### Check that there are still other documents

In [33]:
# Check how many documents remain within the establsihments collection
len(list(establishments.find({})))

38786

## (5) Convert the latitude and longitude to decimal

### Change the data type from String to Decimal for longitude

In [34]:
# Use Update many to change the latitude to decimal
establishments.update_many({}, [ {'$set':{'geocode.latitude': {'$toDecimal': '$geocode.latitude'}}}])

In [36]:
# Check to make sure it worked! 
pprint(establishments.find_one({}, {'geocode'}))

{'_id': ObjectId('63cdeb79a0392a1bef670dad'),
 'geocode': {'latitude': Decimal128('51.083812'), 'longitude': '1.195625'}}


### Change the data type from String to Decimal for latitude

In [37]:
# Change the data type from String to Decimal for longitude
establishments.update_many({}, [ {'$set':{'geocode.longitude': {'$toDecimal': '$geocode.longitude'}}}])

In [ ]:
# Check to make sure it worked! 
pprint(establishments.find_one({}, {'geocode'}))

In [39]:
mongo.close()